In [2]:
# 输入 月份 标签
# params = {
#     "monthyear" :"202504",  # 查数据库中是否有 pay_202504 表格，没有则返回没有该表格，有则返回该表格数据
#     "tag":"N" ,  # 为空则默认查找所有数据， 有则进行筛选
#     "op":"", # 为空则进行查找，有策略名则进行sql操作
#     "key":"kfc", # 为空则查找全部，有则查找 pay_source 或 pay_note 字段含有key的内容
#     "tag_word":"早餐" # 为空则按tag查找，有则按一个映射数组，查出中文对应的tag，比如早餐对应 eat_bf
#     "order":"" # 为空则按 pay_money 降序，1 则按pay——money 升序，2 则筛选pay_money大于100,3 则筛选pay_money 小于30
#     "id":"" # 输入id则排除指定id记录
# }
# 读取列表



# params = {
#     "monthyear" :"202504",  # 查数据库中是否有 pay_202504 表格，没有则返回没有该表格，有则返回该表格数据
#     "tag":"N" ,  # 为空则默认查找所有数据， 有则进行筛选
#     "key":"kfc", # 为空则查找全部，有则查找 pay_source 或 pay_note 字段含有key的内容
#     "lt":"", # pay_money 小于某个金额
#     "gt":"" # pay_money 大于某个金额
# }

# 按关键字搜索
# 批量修改

import sqlite3
from rich.console import Console
from rich.table import Table
from rich.style import Style
from rich.text import Text

def query_payment_data(cursor, params):
    """
    查询支付数据
    
    Args:
        cursor: 数据库游标
        params: 查询参数字典
    
    Returns:
        tuple: (success, result)
        - success: bool, 查询是否成功
        - result: 成功时为数据列表，失败时为错误信息
    """
    try:
        # 检查表是否存在
        table_name = f"pay_{params['monthyear']}"
        cursor.execute("""
            SELECT name FROM sqlite_master 
            WHERE type='table' AND name=?
        """, (table_name,))
        
        if not cursor.fetchone():
            return False, f"表格 {table_name} 不存在"
        
        # 构建查询条件
        conditions = []
        query_params = []
        
        # 标签筛选
        # if params['tag'] and params['tag'] != "N":
        if params['tag'] :
            conditions.append("pay_tag = ?")
            query_params.append(params['tag'])
        
        # 关键词搜索
        if params['key']:
            conditions.append("(pay_source LIKE ? OR pay_note LIKE ?)")
            query_params.extend([f"%{params['key']}%", f"%{params['key']}%"])
        
        # 金额范围筛选
        if params['lt']:
            try:
                lt_value = float(params['lt'])
                conditions.append("pay_money < ?")
                query_params.append(lt_value)
            except ValueError:
                return False, "金额上限格式错误"
        
        if params['gt']:
            try:
                gt_value = float(params['gt'])
                conditions.append("pay_money > ?")
                query_params.append(gt_value)
            except ValueError:
                return False, "金额下限格式错误"
        
        # 构建完整的SQL查询
        query = f"SELECT * FROM {table_name}"
        if conditions:
            query += " WHERE " + " AND ".join(conditions)
        query += " ORDER BY pay_money DESC"
        
        # 执行查询
        cursor.execute(query, query_params)
        
        # 获取列名
        columns = [description[0] for description in cursor.description]
        
        # 将结果转换为字典列表
        results = []
        for row in cursor.fetchall():
            results.append(dict(zip(columns, row)))
        
        return True, results
        
    except sqlite3.Error as e:
        return False, f"数据库错误: {str(e)}"

def display_query_results(results, params):
    """
    使用rich库显示查询结果
    """
    console = Console()
    
    if not results:
        console.print("[red]没有找到数据[/red]")
        return
    
    # 创建表格
    table = Table(
        title=f"支付记录 ({params['monthyear']})",
        show_header=True,
        header_style="bold magenta",
        border_style="blue",
        title_style="bold cyan"
    )
    
    # 添加列
    # table.add_column("ID", style="dim")
    table.add_column("日期", style="cyan")
    table.add_column("金额", justify="right", style="green")
    table.add_column("来源", style="yellow")
    table.add_column("标签", style="red")
    table.add_column("备注")
    
    # 添加数据行
    for row in results:
        money = f"¥{float(row['pay_money']):.2f}"
        
        # 根据金额设置不同样式
        if float(row['pay_money']) > 100:
            money_style = "[bold red]" + money + "[/bold red]"
        else:
            money_style = "[green]" + money + "[/green]"
            
        table.add_row(
            # str(row['id']),
            row['pay_time'],
            money_style,
            row['pay_source'],
            row['pay_tag'],
            row.get('pay_note', '')
        )
    
    # 显示查询条件
    console.print("\n[bold yellow]查询条件:[/bold yellow]")
    conditions = []
    if params['tag'] and params['tag'] != "N":
        conditions.append(f"标签: {params['tag']}")
    if params['key']:
        conditions.append(f"关键词: {params['key']}")
    if params['lt']:
        conditions.append(f"金额 < {params['lt']}")
    if params['gt']:
        conditions.append(f"金额 > {params['gt']}")
    
    if conditions:
        console.print(" | ".join(conditions))
    
    # 打印表格
    console.print("\n")
    console.print(table)
    
    # 打印统计信息
    total_amount = sum(float(row['pay_money']) for row in results)
    console.print(f"\n[bold green]总金额: ¥{total_amount:.2f}[/bold green]")
    console.print(f"[dim]共 {len(results)} 条记录[/dim]")



In [26]:

import sqlite3
from tabulate import tabulate

def update_payment_tags(cursor, params):
    """
    批量更新支付标签
    
    Args:
        cursor: 数据库游标
        params: 更新参数字典
    
    Returns:
        tuple: (success, message)
    """
    try:
        # 标签映射字典
        
        
        # 检查tag_word是否在映射中
        if params['tag_word'] not in tag_mapping:
            return False, f"不支持的标签词: {params['tag_word']}"
            
        new_tag = tag_mapping[params['tag_word']]
        
        # 构建表名
        table_name = f"pay_{params['monthyear']}"
        
        # 检查表是否存在
        cursor.execute("""
            SELECT name FROM sqlite_master 
            WHERE type='table' AND name=?
        """, (table_name,))
        
        if not cursor.fetchone():
            return False, f"表格 {table_name} 不存在"
        
        # 构建查询条件
        conditions = []
        query_params = []
        
        # 关键词搜索
        if params['key']:
            conditions.append("(pay_source LIKE ? OR pay_note LIKE ?)")
            query_params.extend([f"%{params['key']}%", f"%{params['key']}%"])
        
        # 金额范围筛选
        if params['lt']:
            try:
                lt_value = float(params['lt'])
                conditions.append("pay_money < ?")
                query_params.append(lt_value)
            except ValueError:
                return False, "金额上限格式错误"
        
        if params['gt']:
            try:
                gt_value = float(params['gt'])
                conditions.append("pay_money > ?")
                query_params.append(gt_value)
            except ValueError:
                return False, "金额下限格式错误"
        
        # 构建更新语句
        update_sql = f"UPDATE {table_name} SET pay_tag = ?"
        if conditions:
            update_sql += " WHERE " + " AND ".join(conditions)
            
        # 组合参数
        update_params = [new_tag] + query_params
        
        # 执行更新
        cursor.execute(update_sql, update_params)
        
        # 获取更新的行数
        rows_affected = cursor.rowcount
        
        return True, f"成功更新 {rows_affected} 条记录的标签为 {new_tag}"
        
    except sqlite3.Error as e:
        return False, f"数据库错误: {str(e)}"




# up_params={
#     "monthyear": "202501",
#     "key": "手机",
#     "gt": "",
#     "lt": "",
#     "tag_word": ""
# }

# del_params={
#     "include":[]
# }

In [58]:
# 使用更新
# tag_mapping = {

    
#     "早餐": "eat_breakfast",
#     "午餐": "eat_lunch",
#     "晚餐": "eat_dinner",
#     "零食": "eat_snack",
#     "手机": "oline_mobile",
#     "交通": "traffic_ud"
# }

tag_mapping = {
    "外卖": "eat_ol",
    "早餐": "eat_bf",
    "饮料": "eat_dk",
    "出租车": "traffic_dd",
    "地铁": "traffic_ud",
    "充电宝": "traffic_pow",
    "铁路": "tour_high",
    "酒店": "tour_hotel",
    "单车": "traffic_bike",
    "手机": "olpay_phone",
    "软件": "olpay_soft",
    "聚餐": "social_eat",
    "礼物": "social_buy",
    "游戏": "olpay_game",
    "电器": "buy_elec",
    "衣服": "buy_cloth",
    "洗漱": "buy_wash",
    "书": "buy_book",
    "医疗": "buy_medical",
    "生活": "living_use",
    "剪发": "living_hair",
    "快递": "living_delivery",
    "工资": "income_salary",
    "副业": "income_sideline",
    "投资": "income_invest",
    "考试": "invest_exam",
    "上课": "invest_course",
    "房租": "house_rent",
    "会员": "olpay_vip",
    "午餐": "eat_m",
    "红包": "social_hong",
    "娱乐": "social_fun",
    "转账": "social_ex",
    "信用卡": "credit"
}


def main():

    # 创建数据库连接
    conn = sqlite3.connect('moneycount.db')
    cursor = conn.cursor()

    # 更新参数
    up_params = {
        "monthyear": "202504",
        "key": "",
        "gt": "",
        "lt": "",
        "tag_word": ""
    }


    try:
        # 执行更新
        # success, message = update_payment_tags(cursor, up_params)
        # print("\n" + message)

        # 执行常规过滤
        if False:

            # 钱大妈
            up_params["key"] = "钱大妈"
            up_params["tag_word"] = "午餐"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 肯德基
            up_params["key"] = "肯德基"
            up_params["tag_word"] = "午餐"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 美团
            up_params["key"] = "美团"
            up_params["tag_word"] = "午餐"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 地铁
            up_params["key"] = "地铁"
            up_params["tag_word"] = "地铁"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            up_params["key"] = "岭南通"
            up_params["tag_word"] = "地铁"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 铁路
            up_params["key"] = "铁路"
            up_params["tag_word"] = "铁路"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 滴滴
            up_params["key"] = "滴滴"
            up_params["tag_word"] = "出租车"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 手机话费
            up_params["key"] = "手机"
            up_params["tag_word"] = "手机"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 便利店
            up_params["key"] = "便利"
            up_params["tag_word"] = "饮料"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            up_params["key"] = "美宜佳"
            up_params["tag_word"] = "饮料"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            up_params["key"] = "零食"
            up_params["tag_word"] = "饮料"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            up_params["key"] = "蜜雪"
            up_params["tag_word"] = "饮料"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 苹果
            up_params["key"] = "Apple"
            up_params["tag_word"] = "会员"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 转账
            up_params["key"] = "转账"
            up_params["tag_word"] = "转账"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            up_params["key"] = "收款"
            up_params["tag_word"] = "转账"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            up_params["key"] = "发给"
            up_params["tag_word"] = "红包"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 房租
            up_params["key"] = "房租"
            up_params["tag_word"] = "房租"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 剪发
            up_params["key"] = "优剪"
            up_params["tag_word"] = "剪发"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 社保
            up_params["key"] = "医疗"
            up_params["tag_word"] = "医疗"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            up_params["key"] = "医院"
            up_params["tag_word"] = "医疗"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 梯子
            up_params["key"] = "铁臂"
            up_params["tag_word"] = "会员"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 丰巢快递
            up_params["key"] = "丰巢"
            up_params["tag_word"] = "快递"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            up_params["key"] = "顺丰"
            up_params["tag_word"] = "快递"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 京东
            up_params["key"] = "京东"
            up_params["tag_word"] = "生活"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 阿里云
            up_params["key"] = "阿里"
            up_params["tag_word"] = "软件"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # 百度网盘
            up_params["key"] = "百度"
            up_params["tag_word"] = "会员"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

            # steam游戏
            up_params["key"] = "Steam"
            up_params["tag_word"] = "游戏"
            success, message = update_payment_tags(cursor, up_params)
            print("\n" + message)

        # 提交事务
        conn.commit()

    except Exception as e:
        print(f"错误: {str(e)}")
        conn.rollback()

    finally:
        # 关闭连接
        conn.close()


if __name__ == "__main__":
    main()


成功更新 7 条记录的标签为 eat_m

成功更新 9 条记录的标签为 eat_m

成功更新 21 条记录的标签为 eat_m

成功更新 0 条记录的标签为 traffic_ud

成功更新 1 条记录的标签为 traffic_ud

成功更新 0 条记录的标签为 tour_high

成功更新 1 条记录的标签为 traffic_dd

成功更新 1 条记录的标签为 olpay_phone

成功更新 2 条记录的标签为 eat_dk

成功更新 1 条记录的标签为 eat_dk

成功更新 0 条记录的标签为 eat_dk

成功更新 0 条记录的标签为 eat_dk

成功更新 1 条记录的标签为 olpay_vip

成功更新 0 条记录的标签为 social_ex

成功更新 2 条记录的标签为 social_ex

成功更新 0 条记录的标签为 social_hong

成功更新 0 条记录的标签为 house_rent

成功更新 2 条记录的标签为 living_hair

成功更新 1 条记录的标签为 buy_medical

成功更新 1 条记录的标签为 buy_medical

成功更新 1 条记录的标签为 olpay_vip

成功更新 1 条记录的标签为 living_delivery

成功更新 0 条记录的标签为 living_delivery

成功更新 0 条记录的标签为 living_use

成功更新 0 条记录的标签为 olpay_soft

成功更新 0 条记录的标签为 olpay_vip

成功更新 0 条记录的标签为 olpay_game


In [1]:
# 使用查询
def main():
    # 测试参数
    params = {
        "monthyear": "202506",
        "tag": "",
        "key": "",
        "lt": "",
        "gt": ""
    }

    # 创建数据库连接
    # conn = sqlite3.connect('moneycount.db')
    cursor = conn.cursor()

    # 执行查询
    success, result = query_payment_data(cursor, params)

    if success:
        display_query_results(result, params)
    else:
        console = Console()
        console.print(f"[red]错误: {result}[/red]")

    # 关闭连接
    conn.close()


if __name__ == "__main__":
    main()

NameError: name 'conn' is not defined